# Random Walk Simulation and Market Data Analysis

This notebook demonstrates a random walk simulation, analyzes market data, back-tests stock strategies, and applies risk management strategies.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import time
import yfinance as yf
import os

CONFIDENCE_LEVEL = 0.95

## Generating Random Walk

We'll start by generating a random walk with a specified number of steps, mean, and standard deviation.

In [2]:
def generate_random_walk(n=1000, mean=0, std_dev=1):
    return np.cumsum(np.random.normal(mean, std_dev, n))

# Generate random walk
n = 1000
random_walk = generate_random_walk(n=n)
random_walk

## Simulating Multiple Random Walk Paths

Using the generated random walk, we'll simulate multiple paths.

In [3]:
def monte_carlo_simulation(num_simulations=1000, n=1000, mean=0, std_dev=1):
    simulated_series = []
    for _ in tqdm(range(num_simulations), desc="Monte Carlo Simulation"):
        simulated_series.append(generate_random_walk(n, mean, std_dev))
    return np.array(simulated_series)

# Perform Monte Carlo simulation
num_simulations = 1000
simulated_series = monte_carlo_simulation(num_simulations=num_simulations, n=n)
simulated_series

## Plotting the Simulation Results

Finally, we'll plot the results of our Monte Carlo simulation.

In [4]:
def plot_monte_carlo_simulation(simulated_series):
    plt.figure(figsize=(12, 6))
    for simulation in simulated_series:
        plt.plot(simulation, linewidth=0.6, alpha=0.1)
    plt.title('Monte Carlo Simulation of Random Walk')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()

# Plot the results
plot_monte_carlo_simulation(simulated_series)

## Market Data Analysis

We'll fetch market data for a specific stock ticker and analyze it.

In [5]:
def analyze_market_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Returns'] = stock_data['Adj Close'].pct_change().dropna()
    return stock_data

# Example: Analyzing market data for Apple (AAPL)
ticker = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-01-01'
stock_data = analyze_market_data(ticker, start_date, end_date)
stock_data.head()

## Back-Testing Stock Strategies

We'll implement a simple moving average crossover strategy and back-test it on historical data.

In [6]:
def backtest_strategy(ticker, start_date, end_date, short_window=40, long_window=100):
    stock_data = analyze_market_data(ticker, start_date, end_date)
    stock_data['Short_MA'] = stock_data['Adj Close'].rolling(window=short_window, min_periods=1).mean()
    stock_data['Long_MA'] = stock_data['Adj Close'].rolling(window=long_window, min_periods=1).mean()
    stock_data['Signal'] = 0
    stock_data['Signal'][short_window:] = np.where(stock_data['Short_MA'][short_window:] > stock_data['Long_MA'][short_window:], 1, 0)
    stock_data['Position'] = stock_data['Signal'].diff()
    return stock_data

# Example: Back-testing strategy for Apple (AAPL)
backtested_data = backtest_strategy(ticker, start_date, end_date)
backtested_data[['Adj Close', 'Short_MA', 'Long_MA', 'Signal', 'Position']].head()

## Applying Risk Management Strategy

We'll calculate Value at Risk (VaR) and Expected Shortfall (ES) to manage the risk of the portfolio.

In [7]:
def calculate_var_es(returns, confidence_level=0.95):
    var = np.percentile(returns.dropna(), (1-confidence_level)*100)
    es = returns[returns <= var].mean()
    return var, es

# Example: Calculating VaR and ES for Apple (AAPL)
returns = stock_data['Returns']
var, es = calculate_var_es(returns)
var, es

## Running the Entire Analysis

To run the entire analysis, execute the code below which combines all the steps into a single function.

In [8]:
def main(n=1000, mean=0, std_dev=1, confidence_level=CONFIDENCE_LEVEL, output_dir=None, ticker=None, start_date=None, end_date=None, backtest=False):
    """
    Main function to run the analysis.
    """
    if backtest and ticker and start_date and end_date:
        data = backtest_strategy(ticker, start_date, end_date)
    elif ticker and start_date and end_date:
        data = analyze_market_data(ticker, start_date, end_date)
        returns = data['Returns']
        var, es = calculate_var_es(returns, confidence_level)
        print(f"Value at Risk (VaR): {var}")
        print(f"Expected Shortfall (ES): {es}")
    else:
        simulated_series = monte_carlo_simulation(num_simulations=1000, n=n, mean=mean, std_dev=std_dev)
        plot_monte_carlo_simulation(simulated_series)

if __name__ == "__main__":
    main()